In [1]:
#| default_exp store_gsc

In [ ]:
#| export 
from fastapi import FastAPI, HTTPException, Query,APIRouter
from typing import Dict, List, Optional
from datetime import datetime, timedelta
from googleapiclient.discovery import build
import asyncio
from seo_rat.gsc_db import SearchConsoleDB
from seo_rat.fast_gsc import  app


In [ ]:
#| export
store_router = APIRouter(prefix="/store", tags=["Storage"])

In [ ]:
#| export
from seo_rat.fast_gsc import  GSCAuth
auth = GSCAuth()

In [ ]:
#| export
db = SearchConsoleDB()


In [ ]:
#| export
from bson import ObjectId

def serialize_mongodb_doc(doc):
    """Helper function to serialize MongoDB documents"""
    if isinstance(doc, dict):
        return {k: serialize_mongodb_doc(v) for k, v in doc.items()}
    elif isinstance(doc, list):
        return [serialize_mongodb_doc(item) for item in list(doc)]
    elif isinstance(doc, ObjectId):
        return str(doc)
    elif isinstance(doc, datetime):
        return doc.isoformat()
    return doc

In [ ]:
#| export
from pydantic import BaseModel, Field
from datetime import datetime, timedelta
from typing import List, Dict, Any


class StoreAnalyticsResponse(BaseModel):
    """Response model for analytics storage

    Attributes:
        message: Status message
        date: Date of stored data
        dimensions: List of dimensions stored
        records_count: Optional number of records stored
    """

    message: str
    date: str
    dimensions: List[str]
    records_count: Optional[int] = None


class AnalyticsDocument(BaseModel):
    """MongoDB document structure for analytics data

    Attributes:
        site_url: Website URL
        date: Date of data
        dimensions: Dictionary of dimension values
        metrics: Dictionary of metric values
        timestamp: Time of storage
    """

    site_url: str
    date: str
    dimensions: Dict[str, str]
    metrics: Dict[str, float]
    timestamp: datetime


@store_router.get(
    "/store-analytics/{site_url}",
    response_model=StoreAnalyticsResponse,
    summary="Store analytics data",
)
async def store_site_analytics(
    site_url: str,  # Website URL to analyze
    date: str = Query(
        default=datetime.now().strftime("%Y-%m-%d"),
        description="Date to store (YYYY-MM-DD)",
    ),
) -> StoreAnalyticsResponse:
    """Store Google Search Console analytics data with all dimensions.

    Retrieves and stores analytics data for a specific date, including all
    dimension combinations (query, page, country, device). Handles data
    availability delay and prevents duplicate storage.

    Args:
        site_url: Website to analyze
        date: Date to store data for (YYYY-MM-DD)

    Returns:
        StoreAnalyticsResponse containing:
        - Status message
        - Date processed
        - Dimensions stored
        - Number of records stored

    Raises:
        HTTPException:
            - 401 if not authenticated
            - 500 for API or database errors

    Examples:
        >>> # Store today's data (adjusted for delay)
        >>> response = await store_site_analytics(
        ...     "sc-domain:example.com"
        ... )

        >>> # Store data for specific date
        >>> response = await store_site_analytics(
        ...     "sc-domain:example.com",
        ...     date="2024-01-01"
        ... )

    Notes:
        - Google Search Console data has a 3-day delay
        - Maximum 5000 records per request
        - Stores all dimension combinations
        - Prevents duplicate storage for same date
    """
    if not auth.credentials:
        raise HTTPException(
            status_code=401, detail="Not authenticated. Please visit /login first."
        )

    try:
        # Validate and check date availability
        target_date = datetime.strptime(date, "%Y-%m-%d")
        today = datetime.now()
        latest_available = today - timedelta(days=2)

        if target_date.date() >= latest_available.date():
            return StoreAnalyticsResponse(
                message="Data not yet available",
                date=date,
                dimensions=[],
                records_count=0,
            )

        # Initialize service with auth credentials
        service = build("searchconsole", "v1", credentials=auth.credentials)

        # Request data with all dimensions
        response = (
            service.searchanalytics()
            .query(
                siteUrl=site_url,
                body={
                    "startDate": date,
                    "endDate": date,
                    "dimensions": ["query", "page", "country", "device"],
                    "rowLimit": 5000,
                },
            )
            .execute()
        )

        # Process and store data
        if "rows" in response:
            documents = []
            for row in response["rows"]:
                doc = AnalyticsDocument(
                    site_url=site_url,
                    date=date,
                    dimensions={
                        "query": row["keys"][0],
                        "page": row["keys"][1],
                        "country": row["keys"][2],
                        "device": row["keys"][3],
                    },
                    metrics={
                        "clicks": row["clicks"],
                        "impressions": row["impressions"],
                        "ctr": row["ctr"],
                        "position": row["position"],
                    },
                    timestamp=datetime.now(),
                )
                documents.append(doc.dict())

            if documents:
                db.db.analytics.insert_many(documents)

            return StoreAnalyticsResponse(
                message=f"Successfully stored analytics data",
                date=date,
                dimensions=["query", "page", "country", "device"],
                records_count=len(documents),
            )

        return StoreAnalyticsResponse(
            message="No data found for the specified date",
            date=date,
            dimensions=[],
            records_count=0,
        )

    except Exception as e:
        raise HTTPException(
            status_code=500, detail=f"Failed to store analytics data: {str(e)}"
        )


In [ ]:
#| export
from pydantic import BaseModel, Field
from typing import List, Dict, Any
import asyncio


class DateResult(BaseModel):
    """Result for a single date's processing

    Attributes:
        date: Date processed
        records: Number of records stored
    """

    date: str
    records: int


class DateError(BaseModel):
    """Error information for a failed date

    Attributes:
        date: Date that failed
        reason: Error message
    """

    date: str
    reason: str


class RangeAnalyticsResponse(BaseModel):
    """Response model for range analytics storage

    Attributes:
        successful_dates: List of successfully processed dates
        failed_dates: List of dates that failed
        total_records: Total number of records stored
        progress: Current progress information
    """

    successful_dates: List[DateResult]
    failed_dates: List[DateError]
    total_records: int
    progress: Dict[str, Any]


@store_router.get(
    "/store-analytics-range/{site_url}",
    response_model=RangeAnalyticsResponse,
    summary="Store analytics for date range",
)
async def store_site_analytics_range(
    site_url: str,
    start_date: str = Query(
        default="2023-12-29", description="Start date (YYYY-MM-DD)"
    ),
    end_date: str = Query(default="2024-12-06", description="End date (YYYY-MM-DD)"),
) -> RangeAnalyticsResponse:
    """Store Google Search Console analytics data for a date range.

    Retrieves and stores analytics data for each date in the range,
    with progress tracking and error handling. Includes automatic
    date adjustment for data availability delay.

    Args:
        site_url: Website to analyze
        start_date: Start date of range (YYYY-MM-DD)
        end_date: End date of range (YYYY-MM-DD)

    Returns:
        RangeAnalyticsResponse containing:
        - Successfully processed dates
        - Failed dates with reasons
        - Total records stored
        - Progress information

    Raises:
        HTTPException:
            - 401 if not authenticated
            - 500 for API or database errors

    Examples:
        >>> # Store last month's data
        >>> response = await store_site_analytics_range(
        ...     "sc-domain:example.com",
        ...     start_date="2024-01-01",
        ...     end_date="2024-01-31"
        ... )

    Notes:
        - Processes one day at a time
        - Includes 1-second delay between requests
        - Handles rate limits
        - Maximum 5000 records per day
        - Automatically adjusts for 3-day data delay
    """
    if not auth.credentials:
        raise HTTPException(
            status_code=401, detail="Not authenticated. Please visit /login first."
        )

    try:
        # Initialize service with auth credentials
        service = build("searchconsole", "v1", credentials=auth.credentials)

        # Validate dates
        start = datetime.strptime(start_date, "%Y-%m-%d")
        end = datetime.strptime(end_date, "%Y-%m-%d")
        today = datetime.now()
        latest_available = today - timedelta(days=3)

        # Adjust end date for data availability
        if end.date() >= latest_available.date():
            end = latest_available

        # Initialize results
        results = RangeAnalyticsResponse(
            successful_dates=[],
            failed_dates=[],
            total_records=0,
            progress={
                "total_days": (end - start).days + 1,
                "processed_days": 0,
                "current_date": None,
                "percentage_complete": 0,
            },
        )

        # Process each day
        current_date = start
        while current_date <= end:
            date_str = current_date.strftime("%Y-%m-%d")

            # Update progress
            results.progress.update(
                {
                    "current_date": date_str,
                    "processed_days": (current_date - start).days + 1,
                    "percentage_complete": round(
                        ((current_date - start).days + 1)
                        / ((end - start).days + 1)
                        * 100,
                        2,
                    ),
                }
            )

            try:
                # Get data for current date
                response = (
                    service.searchanalytics()
                    .query(
                        siteUrl=site_url,
                        body={
                            "startDate": date_str,
                            "endDate": date_str,
                            "dimensions": ["query", "page", "country", "device"],
                            "rowLimit": 5000,
                        },
                    )
                    .execute()
                )

                if "rows" in response:
                    # Store data
                    documents = [
                        AnalyticsDocument(
                            site_url=site_url,
                            date=date_str,
                            dimensions={
                                "query": row["keys"][0],
                                "page": row["keys"][1],
                                "country": row["keys"][2],
                                "device": row["keys"][3],
                            },
                            metrics={
                                "clicks": row["clicks"],
                                "impressions": row["impressions"],
                                "ctr": row["ctr"],
                                "position": row["position"],
                            },
                            timestamp=datetime.now(),
                        ).dict()
                        for row in response["rows"]
                    ]

                    if documents:
                        # Check for existing data
                        db.db.analytics.delete_many(
                            {"site_url": site_url, "date": date_str}
                        )
                        # Insert new data
                        db.db.analytics.insert_many(documents)
                        results.successful_dates.append(
                            DateResult(date=date_str, records=len(documents))
                        )
                        results.total_records += len(documents)
                    else:
                        results.failed_dates.append(
                            DateError(date=date_str, reason="No data found")
                        )

            except Exception as e:
                results.failed_dates.append(DateError(date=date_str, reason=str(e)))

            # Move to next day
            current_date += timedelta(days=1)
            await asyncio.sleep(1)  # Rate limit delay

        return results

    except Exception as e:
        raise HTTPException(
            status_code=500, detail=f"Failed to store range data: {str(e)}"
        )

In [ ]:
#| export
@store_router.get("/debug-storage/{site_url}")
async def debug_storage(
    site_url: str,
    date: str = Query(default=datetime.now().strftime('%Y-%m-%d'))
):
    """Debug storage for a specific date"""
    try:
        # Add sc-domain: prefix if not present
        if not site_url.startswith('sc-domain:'):
            site_url = f'sc-domain:{site_url}'
            
        # Query for specific date
        date_docs = list(db.db.analytics.find({
            'site_url': site_url,
            'date': date
        }))
        
        # Serialize documents
        serialized_docs = [serialize_mongodb_doc(doc) for doc in date_docs]
        
        return {
            "site_url": site_url,
            "date": date,
            "total_documents": len(serialized_docs),
            "documents": serialized_docs,
            "dimension_types": list(set(doc['dimension_type'] for doc in date_docs)),
            "metrics_summary": {
                "total_clicks": sum(doc['metrics']['clicks'] for doc in date_docs),
                "total_impressions": sum(doc['metrics']['impressions'] for doc in date_docs),
                "average_position": sum(doc['metrics']['position'] for doc in date_docs) / len(date_docs) if date_docs else 0
            }
        }
    except Exception as e:
        print(f"Debug error: {str(e)}")  # Print error for debugging
        raise HTTPException(status_code=500, detail=f"Database error: {str(e)}")

In [ ]:
#| export
from bson import ObjectId
import json

class JSONEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, ObjectId):
            return str(o)
        return super().default(o)

@store_router.get("/debug-mongodb")
async def debug_mongodb():
    """Debug MongoDB contents"""
    try:
        # Get database information
        collections = db.db.list_collection_names()
        
        # Get counts for each collection
        collection_stats = {}
        for collection in collections:
            # Get a sample document and convert ObjectId to string
            sample_doc = db.db[collection].find_one()
            if sample_doc and '_id' in sample_doc:
                sample_doc['_id'] = str(sample_doc['_id'])
            
            collection_stats[collection] = {
                "count": db.db[collection].count_documents({}),
                "sample": sample_doc,
                "unique_sites": list(db.db[collection].distinct("site_url")),
                "unique_dates": list(db.db[collection].distinct("date"))
            }
            
        return {
            "database_name": db.db.name,
            "collections": list(collections),
            "stats": collection_stats
        }
    except Exception as e:
        print(f"Debug error: {str(e)}")  # Print error for debugging
        raise HTTPException(status_code=500, detail=f"MongoDB error: {str(e)}")

In [ ]:
#| export
app.include_router(store_router)
